<img src="fcn_model.png">

Reference 1 : https://github.com/shekkizh/FCN.tensorflow/blob/master/FCN.py
<br>Reference 2 : https://modulabs-biomedical.github.io/FCN
<br>Reference 3 : https://stackoverflow.com/questions/38160940/how-to-count-total-number-of-trainable-parameters-in-a-tensorflow-model

In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.slim as slim

C:\Users\user\Anaconda3\Anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# image size

height = 224
width = 224
num_of_channels = 3
num_of_classes = 21

keep_prob = 0.5

In [3]:
image = tf.placeholder(tf.float32, [None, height, width, num_of_channels])

In [4]:
def fcn8(image, num_of_classes, keep_prob):
    
    "Feature Extraction"
    print("------------------- Conv Layer -------------------")
    # output shape = [None, H/2, H/2, 96]
    conv1 = slim.conv2d(image, 96, [3, 3], scope = 'conv1_1')
    conv1 = slim.conv2d(conv1, 96, [3, 3], scope = 'conv1_2')
    pool1 = slim.max_pool2d(conv1, [2, 2], scope = 'pool1', padding = 'SAME') 
    print(pool1.name + " / shape = " + str(pool1.shape))
    
    # output shape = [None, H/4, H/4, 256]
    conv2 = slim.repeat(pool1, 2, slim.conv2d, 256, [3, 3], scope = 'conv2')
    pool2 = slim.max_pool2d(conv2, [2, 2], scope = 'pool2')
    print(pool2.name + " / shape = " + str(pool2.shape))
    
    # output shape = [None, H/8, H/8, 384]
    conv3 = slim.repeat(pool2, 3, slim.conv2d, 384, [3, 3], scope = 'conv3')
    pool3 = slim.max_pool2d(conv3, [2, 2], scope = 'pool3', padding = 'SAME')
    print(pool3.name + " / shape = " + str(pool3.shape))
    
    # output shape = [None, H/16, H/16, 384]
    conv4 = slim.repeat(pool3, 3, slim.conv2d, 384, [3, 3], scope = 'conv4')
    pool4 = slim.max_pool2d(conv4, [2, 2], scope = 'pool4', padding = 'SAME')
    print(pool4.name + " / shape = " + str(pool4.shape))
    
    # output shape = [None, H/32, H/32, 256]
    conv5 = slim.repeat(pool4, 3, slim.conv2d, 256, [3, 3], scope = 'conv5')
    pool5 = slim.max_pool2d(conv5, [2, 2], scope = 'pool5', padding = 'SAME')
    print(pool5.name + " / shape = " + str(pool5.shape))
    
    
    "Feature-level Classificaiton"
    # output shape = [None, H/32, H/32, 4096]
    conv6 = slim.conv2d(pool5, 4096, [7, 7], scope = 'conv6')
    conv6 = tf.nn.dropout(conv6, keep_prob = keep_prob)
    print(conv6.name + " / shape = " + str(conv6.shape))
    
    # output shape = [None, H/32, H/32, 4096]
    conv7 = slim.conv2d(conv6, 4096, [1, 1], scope = 'conv7')
    conv7 = tf.nn.dropout(conv7, keep_prob = keep_prob)
    print(conv7.name + " / shape = " + str(conv7.shape))
    
    # output shape = [None, H/32, H/32, num_of_classes]
    conv8 = slim.conv2d(conv7, num_of_classes, [1, 1], scope = 'conv8')
    print(conv8.name + " / shape = " + str(conv8.shape))
    
    
    "Upsampling"
    print("------------------- Upsampling -------------------")
    conv_t1 = slim.conv2d_transpose(conv8, num_outputs = pool4.get_shape()[3], kernel_size = [4, 4], stride = 2)
    fuse_1 = tf.add(conv_t1, pool4, name = 'fuse_1')
    print(fuse_1.name + " / shape = " + str(fuse_1.shape))
    
    conv_t2 = slim.conv2d_transpose(fuse_1, num_outputs = pool3.get_shape()[3], kernel_size = 4, stride = 2) 
    fuse_2 = tf.add(conv_t2, pool3, name = 'fuse_2')
    print(fuse_2.name + " / shape = " + str(fuse_2.shape))
    
    conv_t3 = slim.conv2d_transpose(fuse_2, num_outputs = num_of_channels, kernel_size = 16, stride = 8)
    print(conv_t3.name + " / shape = " + str(conv_t3.shape))
    
    
    "Segmentation"
    print("------------------ Segmentation ------------------")
    annotation_pred = tf.argmax(conv_t3, dimension = num_of_channels, name="prediction")
    print(annotation_pred.name + " / shape = " + str(annotation_pred.shape))
    fcn8 = tf.expand_dims(annotation_pred, dim = num_of_channels)
    print(fcn8.name + " / shape = " + str(fcn8.shape))
    
    return fcn8

In [5]:
fcn8 = fcn8(image, num_of_classes, keep_prob)

------------------- Conv Layer -------------------
pool1/MaxPool:0 / shape = (?, 112, 112, 96)
pool2/MaxPool:0 / shape = (?, 56, 56, 256)
pool3/MaxPool:0 / shape = (?, 28, 28, 384)
pool4/MaxPool:0 / shape = (?, 14, 14, 384)
pool5/MaxPool:0 / shape = (?, 7, 7, 256)
dropout/mul:0 / shape = (?, 7, 7, 4096)
dropout_1/mul:0 / shape = (?, 7, 7, 4096)
conv8/Relu:0 / shape = (?, 7, 7, 21)
------------------- Upsampling -------------------
fuse_1:0 / shape = (?, 14, 14, 384)
fuse_2:0 / shape = (?, 28, 28, 384)
Conv2d_transpose_2/Relu:0 / shape = (?, 224, 224, 3)
------------------ Segmentation ------------------
prediction:0 / shape = (?, 224, 224)
ExpandDims:0 / shape = (?, 224, 224, 1)


In [7]:
total_parameters = np.sum([np.prod(var.get_shape().as_list()) for var in tf.trainable_variables()])
print("Number of Weights : " + format(total_parameters, ','))

Number of Weights : 81,520,632
